In [1]:
import ast
import pickle
import re

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [2]:
#data = json.load(open('C:/Users/sudsaha1/Documents/My Received Files/TransformedSummary.json'))

#TransactionDf = [item for items in data.values() for item in items]
#TransactionDf=pd.DataFrame(TransactionDf) 
TransactionDf = pd.read_csv('C:/Users/sudsaha1/Documents/My Received Files/CombinedSummary.csv')

In [3]:
TransactionDf.head()

,Unnamed: 0,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime,category,Detailed Category
0,296,32353546,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-11-29T20:21:50-08:00,Credit,Booked,100223,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC",2016-11-29T20:21:50-08:00,Services,Health
1,76,32353530,"{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-12-05T20:21:50-08:00,Credit,Booked,100315,"/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC",2016-12-05T20:21:50-08:00,Services,Health
2,82,32353542,"{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-12-06T20:21:50-08:00,Credit,Booked,100321,"/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC",2016-12-06T20:21:50-08:00,Services,Health
3,268,32353522,"{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2017-01-11T20:21:50-08:00,Credit,Booked,100401,"/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC",2017-01-11T20:21:50-08:00,Services,Health
4,269,32353524,"{""Amount"": ""45"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2017-01-24T20:21:50-08:00,Credit,Booked,100402,"/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC",2017-01-24T20:21:50-08:00,Services,Health


In [4]:
TransactionDf['AmountInGBP'] = TransactionDf['Amount'].map(lambda x: ast.literal_eval(x)['Amount']).astype(np.float)
TransactionDf['Type'] = TransactionDf['BankTransactionCode'].map(lambda x: ast.literal_eval(x[:-1] + "\"\"" + '}')['Code'])

In [5]:
TransactionDf.head().transpose()

,0,1,2,3,4
Unnamed: 0,296,76,82,268,269
AccountId,32353546,32353530,32353542,32353522,32353524
Amount,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Amount"": ""45"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }"
BookingDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00
CreditDebitIndicator,Credit,Credit,Credit,Credit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100223,100315,100321,100401,100402
TransactionInformation,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC"
ValueDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00


In [6]:
def BinningData(TransactionDf, bins):
    min_val = min((TransactionDf['AmountInGBP']) - 1)
    max_val = max(TransactionDf['AmountInGBP'])
    # print(min_val, max_val)
    custom_bucket_array = np.linspace(min_val, max_val, bins)
    cut_points = list(custom_bucket_array)
    group_name = ["low", "medium", "high"]
    TransactionDf["AmountInGBP_bin"] = pd.cut(TransactionDf["AmountInGBP"], cut_points, labels=group_name)
    return TransactionDf


BinningData(TransactionDf, 4)

,Unnamed: 0,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime,category,Detailed Category,AmountInGBP,Type,AmountInGBP_bin
0,296,32353546,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-11-29T20:21:50-08:00,Credit,Booked,100223,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC",2016-11-29T20:21:50-08:00,Services,Health,139.90,CHP,low
1,76,32353530,"{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-12-05T20:21:50-08:00,Credit,Booked,100315,"/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC",2016-12-05T20:21:50-08:00,Services,Health,388.50,CHP,low
2,82,32353542,"{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2016-12-06T20:21:50-08:00,Credit,Booked,100321,"/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC",2016-12-06T20:21:50-08:00,Services,Health,50.40,CHP,low
3,268,32353522,"{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2017-01-11T20:21:50-08:00,Credit,Booked,100401,"/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC",2017-01-11T20:21:50-08:00,Services,Health,404.00,CHP,low
4,269,32353524,"{""Amount"": ""45"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2017-01-24T20:21:50-08:00,Credit,Booked,100402,"/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC",2017-01-24T20:21:50-08:00,Services,Health,45.00,CHP,low
5,322,32353564,"{""Amount"": ""45"", ""Currency"": ""GBP""}","{""Code"": ""CHP"","" SubCode"": }",2017-02-15T20:21:50-08:00,Debit,Booked,100522,"/RFB/GMCI STATMT , PCM55CI73801766 , HENNER GMC",2017-02-15T20:21:50-08:00,Services,Health,45.00,CHP,low
6,280,32353542,"{""Amount"": ""88.44"", ""Currency"": "" GBP""}","{""Code"": ""CHP"","" SubCode"": }",2017-11-23T20:21:50-08:00,Credit,Booked,101611,"/RFB/GMCI STATMT , PCM55CI78411388 , HENNER GMC",2017-11-23T20:21:50-08:00,Services,Health,88.44,CHP,low
7,214,32353524,"{""Amount"": ""5"", ""Currency"": "" GBP""}","{""Code"": ""POS"","" SubCode"": }",2017-08-02T20:21:50-08:00,Debit,Booked,101302,"1177 01AUG17 CD , NISA VIRGINIA , QUAY , LOND...",2017-08-04T20:21:50-08:00,Retail,Supermarket,5.00,POS,low
8,339,32353526,"{""Amount"": ""0.99"", ""Currency"": "" GBP""}","{""Code"": ""POS"","" SubCode"": }",2017-07-03T20:21:50-08:00,Debit,Booked,101203,"1177 01JUL17 CD , TESCO STORES 6554 , LONDON GB",2017-07-03T20:21:50-08:00,Retail,Supermarket,0.99,POS,low
9,215,32353526,"{""Amount"": ""1"", ""Currency"": "" GBP""}","{""Code"": ""POS"","" SubCode"": }",2017-08-03T20:21:50-08:00,Debit,Booked,101303,"1177 02AUG17 CD , TESCO STORES 6554 , LONDON GB",2017-08-04T20:21:50-08:00,Retail,Supermarket,1.00,POS,low


In [7]:
TransactionDf.head().transpose()

,0,1,2,3,4
Unnamed: 0,296,76,82,268,269
AccountId,32353546,32353530,32353542,32353522,32353524
Amount,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Amount"": ""45"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }"
BookingDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00
CreditDebitIndicator,Credit,Credit,Credit,Credit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100223,100315,100321,100401,100402
TransactionInformation,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC"
ValueDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00


In [8]:
# Get Merchant info from Description
def getMerchfromDesc(tranType, tranDesc):
    if tranType == 'POS':
        parsed = tranDesc.split(', ')
        lenPar = len(parsed)
        if parsed[lenPar - 1] == 'REFUND':
            merch = parsed[1:-2]
            return ('').join(merch) + 'REFUND'
        else:
            merch = parsed[1:-1]
            return ('').join(merch)
    else:
        return tranDesc


# Clean Merchant info
def cleanMerchant(txt):
    txt = re.sub(r'[^a-zA-Z\b]', ' ', str(txt).upper())
    return " ".join(txt.split())

In [9]:
TransactionDf['Merchant'] = TransactionDf.apply(
    lambda row: getMerchfromDesc(row['Type'], row['TransactionInformation']), axis=1)
TransactionDf['cleanedMerchant'] = TransactionDf['Merchant'].map(lambda x: cleanMerchant(x))

TransactionDf['ConcatCategory'] = TransactionDf['category'] + ', ' + TransactionDf['Detailed Category']
# len(TransactionDf['ConcatCategory'].unique())

TransactionDf['Predictorcolumn'] = TransactionDf['CreditDebitIndicator'] + ' ' + TransactionDf['cleanedMerchant'] + ' ' + TransactionDf['Type'] + ' ' + TransactionDf['AmountInGBP_bin'].astype(str)

In [10]:
TransactionDf.head().transpose()

,0,1,2,3,4
Unnamed: 0,296,76,82,268,269
AccountId,32353546,32353530,32353542,32353522,32353524
Amount,"{""Amount"": ""139.9"", ""Currency"": ""GBP""}","{""Amount"": ""388.5"", ""Currency"": ""GBP""}","{""Amount"": ""50.4"", ""Currency"": ""GBP""}","{""Amount"": ""404"", ""Currency"": ""GBP""}","{""Amount"": ""45"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }","{""Code"": ""CHP"","" SubCode"": }"
BookingDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00
CreditDebitIndicator,Credit,Credit,Credit,Credit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100223,100315,100321,100401,100402
TransactionInformation,"/RFB/GMCI STATMT , PCM55CI72578736 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72684987 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI72706905 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73243020 , HENNER GMC","/RFB/GMCI STATMT , PCM55CI73446179 , HENNER GMC"
ValueDateTime,2016-11-29T20:21:50-08:00,2016-12-05T20:21:50-08:00,2016-12-06T20:21:50-08:00,2017-01-11T20:21:50-08:00,2017-01-24T20:21:50-08:00


In [11]:
vectorizer = TfidfVectorizer(use_idf=True, stop_words='english', ngram_range=(1, 2), min_df=1)

In [17]:
vectors = vectorizer.fit_transform(TransactionDf['Predictorcolumn'])

AttributeError: 'DataFrame' object has no attribute 'rdd'

In [13]:
help(vectorizer)

Help on TfidfVectorizer in module sklearn.feature_extraction.text object:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be the sequence strings or
 |      bytes items are expected to be analyzed directly.
 |  
 |  encoding : string, 'utf-8' by default.
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 

In [15]:
vectors.show()

AttributeError: show not found

In [ ]:
#with open('../../target/model/vectorizer.pkl', 'wb') as fin:

# open a file, where you ant to store the data
file = open('vectorizer.pkl', 'wb')
pickle.dump(vectorizer,file)

In [ ]:
# clf = RandomForestClassifier(n_estimators=50, max_depth=None,min_samples_split=2, random_state=0)
# scores = cross_val_score(clf, vectors, TransactionDf['ConcatCategory'], cv=3)
# np.mean(scores)

# Training on complete data
RandomForest_model = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=2, random_state=0)
RandomForest_model.fit(vectors, TransactionDf['ConcatCategory'])

In [ ]:

# Dump the trained decision tree classifier with Pickle
RandomForest_pkl_filename = 'RandomForest_classifier.pkl'
# Open the file to save as pkl file
RandomForest_model_pkl = open(RandomForest_pkl_filename, 'wb')
pickle.dump(RandomForest_model, RandomForest_model_pkl)
# Close the pickle instances
RandomForest_model_pkl.close()